In [1]:
using Glob  # find files with a specific pattern
using DelimitedFiles, DataFrames, EzXML #read txt  and xml and then convert them to DataFrames
using Dates
using LaTeXStrings
#using ContinuousWavelets  #wavelet power
using Interpolations # interpolation
#using StaticArrays #faster array
#using VMD # VMD decomposition
using DSP
using LinearAlgebra
using Wavelets
using FFTW
using Statistics
using CSV
include("TCWavelet.jl")

chisquare_solve (generic function with 1 method)

In [ ]:
# date = DateTime(2022, 08, 24, 07, 40, 00)
date = DateTime(2022, 08, 24, 07, 30, 10)
# shock_time = DateTime(2022, 08, 24, 07, 55, 20)
shock_time = DateTime(2022, 08, 24, 07, 55, 20)
ds = 2 

In [ ]:
# read data 
datapath2c1hz = "E:/WorkSpace/Tianwen/Data/MAVEN"
datestr = Dates.format.(date, "yyyymmdd")


In [9]:
println("Amplitude: ", Amplitude)
println("Propagation Angle: ", ProAngle)
println("Shock Angle: ", ShockAngle)

Amplitude: 3.0343390899281153
Propagation Angle: 73.79820229323984
Shock Angle: 123.9626326849692
